In [1]:
### polyphase channelizer
import numpy as np
from scipy.signal import firwin, lfilter, freqz
from scipy.signal import butter, filtfilt, resample
from scipy.fft import fft, ifft, fftshift

import plotly.graph_objects as go
import plotly.express as px

In [2]:
def butter_lowpass_filter(data, cutoff, fs, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)  # zero-phase filtering
    return y

In [3]:
x_f_1 = np.zeros(1000) + 0j
x_f_1[0:400] = 1 + 0j
x_f_1[-400:] = 1 + 0j
x_f_2 = np.zeros(1000) + 0j
x_f_2[0:400] = 1 + 0j
x_f_2[-400:] = 1 + 0j
x_f_3 = np.zeros(1000) + 0j
x_f_3[0:400] = 1 + 0j
x_f_3[-400:] = 1 + 0j
x_f_4 = np.zeros(1000) + 0j
x_f_4[0:400] = 1 + 0j
x_f_4[-400:] = 1 + 0j

x_t_1 = ifft(x_f_1)*len(x_f_1)
x_t_2 = ifft(x_f_2)*len(x_f_2)
x_t_3 = ifft(x_f_3)*len(x_f_3)
x_t_4 = ifft(x_f_4)*len(x_f_4)

In [4]:
x_f_1 = np.random.randn(1000) + 0j
x_f_1[100:900] *= 1e-6
x_f_1_original = x_f_1.copy()
x_t_1 = ifft(x_f_1)*len(x_f_1)
x_t_1 = resample(x_t_1, 4000)

x_f_2 = np.random.randn(1000) + 0j
x_f_2[200:800] *= 1e-6
x_t_2 = ifft(x_f_2)*len(x_f_2)
x_f_2_original = x_f_2.copy()
x_t_2 = resample(x_t_2, 4000)
x_t_2 *= np.exp(1j*2*np.pi*1000*np.arange(len(x_t_2))/4000)

x_f_3 = np.random.randn(1000) + 0j
x_f_3[300:700] *= 1e-6
x_t_3 = ifft(x_f_3)*len(x_f_3)
x_f_3_original = x_f_3.copy()
x_t_3 = resample(x_t_3, 4000)
x_t_3 *= np.exp(1j*2*np.pi*2000*np.arange(len(x_t_3))/4000)

x_f_4 = np.random.randn(1000) + 0j
x_f_4[400:600] *= 1e-6
x_t_4 = ifft(x_f_4)*len(x_f_4)
x_f_4_original = x_f_4.copy()
x_t_4 = resample(x_t_4, 4000)
x_t_4 *= np.exp(1j*2*np.pi*3000*np.arange(len(x_t_4))/4000)

x_f_1 = fft(x_t_1)/len(x_t_1)
x_f_2 = fft(x_t_2)/len(x_t_1)
x_f_3 = fft(x_t_3)/len(x_t_1)
x_f_4 = fft(x_t_4)/len(x_t_1)

x_f_total = x_f_1 + x_f_2 + x_f_3 + x_f_4

x_t_total = ifft(x_f_total)*len(x_f_total)


In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_1)), mode='lines', name=f'Signal 1 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_2)), mode='lines', name=f'Signal 2 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_3)), mode='lines', name=f'Signal 3 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_4)), mode='lines', name=f'Signal 4 Contribution'))
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_total)), mode='lines', name=f'Cmobined Wide-band Signal', line=dict(dash="dot")))    
fig.update_layout(width=800,height=400, title='Multi-Channel  Waveform', xaxis_title='Frequency Sample', yaxis_title='Magnitude (dB)',
                  xaxis_range=[0, 4000], yaxis_range=[-150, 10])

In [8]:
fig.write_html("wideband_signal.html",
               include_plotlyjs='cdn',
               full_html=False)

# Conventional Approach

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

num_taps = 64  # Number of filter taps (coefficients) - typically odd for linear phase

filter_taps = signal.firwin(num_taps, 0.25, pass_zero=True)

filter_freq_response = fft(filter_taps, 4000)

In [15]:
fig_filter = go.Figure()
fig_filter.add_trace(go.Scatter(y=filter_taps, mode='lines+markers', name=f'64-Tap Low-pass Filter'))   
fig_filter.update_layout(width=800,height=400, title='64-Tap Low-pass Filter', xaxis_title='Time Sample', yaxis_title='Magnitude (Linear)',
                  )

In [16]:
fig_filter.write_html("lpf_taps.html",
                        include_plotlyjs='cdn',
                        full_html=False)

In [17]:
### Apply frequency shift on channel 3
x_f_total_shifted = np.roll(x_f_total, -2000)
x_f_1_shifted = np.roll(x_f_1, -2000)
x_f_2_shifted = np.roll(x_f_2, -2000)
x_f_3_shifted = np.roll(x_f_3, -2000)
x_f_4_shifted = np.roll(x_f_4, -2000)

In [20]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_1_shifted)), mode='lines', name=f'Signal 1 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_2_shifted)), mode='lines', name=f'Signal 2 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_3_shifted)), mode='lines', name=f'Signal 3 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_4_shifted)), mode='lines', name=f'Signal 4 Contribution'))
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_total_shifted)), mode='lines', name=f'Wide-band Signal (Frequency-Shifted)', line=dict(dash="dot")))   
fig.update_layout(width=800,height=400, title='Frequency-Shifted Multi-Channel Waveform', xaxis_title='Frequency Sample', yaxis_title='Magnitude (dB)',
                  xaxis_range=[0, 4000], yaxis_range=[-150, 10])

In [21]:
fig.write_html("shifted_wideband_signal.html",
               include_plotlyjs='cdn',
               full_html=False)

In [22]:
# 4. Apply the filter to the signal
x_f_total_filtered = filter_freq_response*x_f_total_shifted
x_f_1_filtered = filter_freq_response*x_f_1_shifted
x_f_2_filtered = filter_freq_response*x_f_2_shifted
x_f_3_filtered = filter_freq_response*x_f_3_shifted
x_f_4_filtered = filter_freq_response*x_f_4_shifted

In [25]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_1_filtered)), mode='lines', name=f'Signal 1 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_2_filtered)), mode='lines', name=f'Signal 2 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_3_filtered)), mode='lines', name=f'Signal 3 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_4_filtered)), mode='lines', name=f'Signal 4 Contribution'))
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_total_filtered)), mode='lines', name=f'Wide-band Signal (Filtered)', line=dict(dash="dot")))   
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(filter_freq_response)), mode='lines', name=f'Filter Response', line=dict(dash="dot")))   
fig.update_layout(width=800,height=400, title='Filtered Multi-Channel Waveform', xaxis_title='Frequency Sample', yaxis_title='Magnitude (dB)',
                  xaxis_range=[0, 4000], yaxis_range=[-150, 10])

In [26]:
fig.write_html("filtered_wideband_signal.html",
               include_plotlyjs='cdn',
               full_html=False)

In [27]:
x_t_total_filtered = ifft(x_f_total_filtered)*len(x_f_total_filtered)
x_t_total_filtered_downsamp = x_t_total_filtered[::4]
x_f_total_filtered_downsamp = fft(x_t_total_filtered_downsamp)/len(x_t_total_filtered_downsamp)

In [31]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_total_filtered_downsamp)), mode='lines', name=f'Signal 3 (Extracted)'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_3_original)), mode='lines', name=f'Signal 3 (Original)'))    
fig.update_layout(width=800,height=400, title='Signal 3 Extracted by Decimation Filtering', xaxis_title='Frequency Sample', yaxis_title='Magnitude (dB)',
                  xaxis_range=[0, 1000], yaxis_range=[-150, 10])

In [32]:
fig.write_html("decim_filtered_signal.html",
               include_plotlyjs='cdn',
               full_html=False)

# Polyphase Channelizer

In [33]:
x_f_1 = np.ones(1000) + 0j
x_f_1[100:900] *= 1e-6
x_f_1_original = x_f_1.copy()
x_t_1 = ifft(x_f_1)*len(x_f_1)
x_t_1 = resample(x_t_1, 4000)

x_f_2 = np.ones(1000) + 0j
x_f_2[200:800] *= 1e-6
x_t_2 = ifft(x_f_2)*len(x_f_2)
x_f_2_original = x_f_2.copy()
x_t_2 = resample(x_t_2, 4000)
x_t_2 *= np.exp(1j*2*np.pi*1000*np.arange(len(x_t_2))/4000)

x_f_3 = np.ones(1000) + 0j
x_f_3[300:700] *= 1e-6
x_t_3 = ifft(x_f_3)*len(x_f_3)
x_f_3_original = x_f_3.copy()
x_t_3 = resample(x_t_3, 4000)
x_t_3 *= np.exp(1j*2*np.pi*2000*np.arange(len(x_t_3))/4000)

x_f_4 = np.ones(1000) + 0j
x_f_4[400:600] *= 1e-6
x_t_4 = ifft(x_f_4)*len(x_f_4)
x_f_4_original = x_f_4.copy()
x_t_4 = resample(x_t_4, 4000)
x_t_4 *= np.exp(1j*2*np.pi*3000*np.arange(len(x_t_4))/4000)

x_f_1 = fft(x_t_1)/len(x_t_1)
x_f_2 = fft(x_t_2)/len(x_t_1)
x_f_3 = fft(x_t_3)/len(x_t_1)
x_f_4 = fft(x_t_4)/len(x_t_1)

x_f_total = x_f_1 + x_f_2 + x_f_3 + x_f_4

x_t_total = ifft(x_f_total)*len(x_f_total)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_1)), mode='lines', name=f'Signal 1 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_2)), mode='lines', name=f'Signal 2 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_3)), mode='lines', name=f'Signal 3 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_4)), mode='lines', name=f'Signal 4 Contribution'))
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_total)), mode='lines', name=f'Wideband Signal', line=dict(dash="dot")))    
fig.update_layout(width=600,height=400, title='Multi-Channel  Waveform', xaxis_title='Frequency Sample', yaxis_title='Magnitude (dB)',
                  xaxis_range=[0, 4000], yaxis_range=[-150, 10])

In [ ]:
### downsample into 4 staggered streams
# stream 1
x_f_total[abs(x_f_total)<1e-3] = np.nan
x_f_1[abs(x_f_1)<1e-3] = np.nan
x_f_2[abs(x_f_2)<1e-3] = np.nan
x_f_3[abs(x_f_3)<1e-3] = np.nan
x_f_4[abs(x_f_4)<1e-3] = np.nan

idx = np.arange(4000)


fig1 = go.Figure()
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_1),
                           z=np.imag(x_f_1),
                           mode='markers', name=f'Channel 1 Content in Stream 1 (delay=0)',
                           marker=dict(size=5, symbol="circle")))    
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_2),
                           z=np.imag(x_f_2),
                           mode='markers', name=f'Channel 2 Content in Stream 1 (delay=0)',
                           marker=dict(size=5, symbol="square")))  
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_3),
                           z=np.imag(x_f_3),
                           mode='markers', name=f'Channel 3 Content in Stream 1 (delay=0)',
                           marker=dict(size=5, symbol="diamond")))  
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_4),
                           z=np.imag(x_f_4),
                           mode='markers', name=f'Channel 4 Content in Stream 1 (delay=0)',
                           marker=dict(size=5, symbol="cross")))


fig1.update_layout(width=1000,height=800,
    scene=dict(
        xaxis=dict( title="Frequency Samples"),
        yaxis=dict(range=[-1.5, 1.5], title="Y"),
        zaxis=dict(range=[-1.5, 1.5], title="Z"),
        aspectmode="manual",           # allow manual control
        aspectratio=dict(x=1, y=0.5, z=0.5),  # scale of each axis
        camera=dict(eye=dict(x=-1, y=-1, z=1)),  # adjust the camera so labels are visible

    ),
    title="Decomposed Spectrum of Stream 1 (delay=0)"
)

# Zero axes

fig1.add_trace(go.Scatter3d(
    x=[0,1000], y=[0,0], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[0,0], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[0,0], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[4000,4000], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[4000,4000], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))


fig1.show()

In [ ]:
### downsample into 4 staggered streams
# stream 1
idx = np.arange(1000)
x_t_total_downsamp_1 = x_t_total[0::4]
x_t_1_downsamp_1 = x_t_1[0::4]
x_t_2_downsamp_1 = x_t_2[0::4]
x_t_3_downsamp_1 = x_t_3[0::4]
x_t_4_downsamp_1 = x_t_4[0::4]

x_f_total_downsamp_1 = fft(x_t_total_downsamp_1)/len(x_t_total_downsamp_1)
x_f_1_downsamp_1 = fft(x_t_1_downsamp_1)/len(x_t_1_downsamp_1)
x_f_2_downsamp_1 = fft(x_t_2_downsamp_1)/len(x_t_2_downsamp_1)
x_f_3_downsamp_1 = fft(x_t_3_downsamp_1)/len(x_t_3_downsamp_1)
x_f_4_downsamp_1 = fft(x_t_4_downsamp_1)/len(x_t_4_downsamp_1)

x_f_total_downsamp_1[abs(x_f_total_downsamp_1)<1e-3] = np.nan
x_f_1_downsamp_1[abs(x_f_1_downsamp_1)<1e-3] = np.nan
x_f_2_downsamp_1[abs(x_f_2_downsamp_1)<1e-3] = np.nan
x_f_3_downsamp_1[abs(x_f_3_downsamp_1)<1e-3] = np.nan
x_f_4_downsamp_1[abs(x_f_4_downsamp_1)<1e-3] = np.nan


fig1 = go.Figure()
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_1_downsamp_1),
                           z=np.imag(x_f_1_downsamp_1),
                           mode='markers', name=f'Channel 1 Content in Stream 1 (delay=0)',
                           marker=dict(size=5, symbol="circle")))    
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_2_downsamp_1),
                           z=np.imag(x_f_2_downsamp_1),
                           mode='markers', name=f'Channel 2 Content in Stream 1 (delay=0)',
                           marker=dict(size=5, symbol="square")))  
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_3_downsamp_1),
                           z=np.imag(x_f_3_downsamp_1),
                           mode='markers', name=f'Channel 3 Content in Stream 1 (delay=0)',
                           marker=dict(size=5, symbol="diamond")))  
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_4_downsamp_1),
                           z=np.imag(x_f_4_downsamp_1),
                           mode='markers', name=f'Channel 4 Content in Stream 1 (delay=0)',
                           marker=dict(size=5, symbol="cross")))


fig1.update_layout(width=1000,height=800,
    scene=dict(
        xaxis=dict( title="Frequency Samples"),
        yaxis=dict(range=[-1.5, 1.5], title="Y"),
        zaxis=dict(range=[-1.5, 1.5], title="Z"),
        aspectmode="manual",           # allow manual control
        aspectratio=dict(x=1, y=0.5, z=0.5),  # scale of each axis
        camera=dict(eye=dict(x=-1, y=-1, z=1)),  # adjust the camera so labels are visible

    ),
    title="Decomposed Spectrum of Stream 1 (delay=0)"
)

# Zero axes

fig1.add_trace(go.Scatter3d(
    x=[0,1000], y=[0,0], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[0,0], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[0,0], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[1000,1000], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[1000,1000], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))


fig1.show()

In [ ]:
### downsample into 4 staggered streams
# stream 1
idx = np.arange(1000)
x_t_total_downsamp_1 = x_t_total[1::4]
x_t_1_downsamp_1 = x_t_1[1::4]
x_t_2_downsamp_1 = x_t_2[1::4]
x_t_3_downsamp_1 = x_t_3[1::4]
x_t_4_downsamp_1 = x_t_4[1::4]

x_f_total_downsamp_1 = fft(x_t_total_downsamp_1)/len(x_t_total_downsamp_1)
x_f_1_downsamp_1 = fft(x_t_1_downsamp_1)/len(x_t_1_downsamp_1)
x_f_2_downsamp_1 = fft(x_t_2_downsamp_1)/len(x_t_2_downsamp_1)
x_f_3_downsamp_1 = fft(x_t_3_downsamp_1)/len(x_t_3_downsamp_1)
x_f_4_downsamp_1 = fft(x_t_4_downsamp_1)/len(x_t_4_downsamp_1)

x_f_total_downsamp_1[abs(x_f_total_downsamp_1)<1e-3] = np.nan
x_f_1_downsamp_1[abs(x_f_1_downsamp_1)<1e-3] = np.nan
x_f_2_downsamp_1[abs(x_f_2_downsamp_1)<1e-3] = np.nan
x_f_3_downsamp_1[abs(x_f_3_downsamp_1)<1e-3] = np.nan
x_f_4_downsamp_1[abs(x_f_4_downsamp_1)<1e-3] = np.nan


fig2 = go.Figure()
fig2.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_1_downsamp_1),
                           z=np.imag(x_f_1_downsamp_1),
                           mode='markers', name=f'Channel 1 Content in Stream 2 (delay=1)',
                           marker=dict(size=5, symbol="circle")))    
fig2.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_2_downsamp_1),
                           z=np.imag(x_f_2_downsamp_1),
                           mode='markers', name=f'Channel 2 Content in Stream 2 (delay=1)',
                           marker=dict(size=5, symbol="square")))  
fig2.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_3_downsamp_1),
                           z=np.imag(x_f_3_downsamp_1),
                           mode='markers', name=f'Channel 3 Content in Stream 2 (delay=1)',
                           marker=dict(size=5, symbol="diamond")))  
fig2.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_4_downsamp_1),
                           z=np.imag(x_f_4_downsamp_1),
                           mode='markers', name=f'Channel 4 Content in Stream 2 (delay=1)',
                           marker=dict(size=5, symbol="cross")))  


fig2.update_layout(width=1000,height=800,
    scene=dict(
        xaxis=dict( title="Frequency Samples"),
        yaxis=dict(range=[-1.5, 1.5], title="Y"),
        zaxis=dict(range=[-1.5, 1.5], title="Z"),
        aspectmode="manual",           # allow manual control
        aspectratio=dict(x=1, y=0.5, z=0.5),  # scale of each axis
        camera=dict(eye=dict(x=-1, y=-1, z=1)),  # adjust the camera so labels are visible

    ),
    title="Decomposed Spectrum of Stream 2 (delay=1)"
)

# Zero axes

fig2.add_trace(go.Scatter3d(
    x=[0,1000], y=[0,0], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig2.add_trace(go.Scatter3d(
    x=[0,0], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig2.add_trace(go.Scatter3d(
    x=[0,0], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig2.add_trace(go.Scatter3d(
    x=[1000,1000], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig2.add_trace(go.Scatter3d(
    x=[1000,1000], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))


fig2.show()

In [ ]:
### downsample into 4 staggered streams
# stream 1
idx = np.arange(1000)
x_t_total_downsamp_1 = x_t_total[2::4]
x_t_1_downsamp_1 = x_t_1[2::4]
x_t_2_downsamp_1 = x_t_2[2::4]
x_t_3_downsamp_1 = x_t_3[2::4]
x_t_4_downsamp_1 = x_t_4[2::4]

x_f_total_downsamp_1 = fft(x_t_total_downsamp_1)/len(x_t_total_downsamp_1)
x_f_1_downsamp_1 = fft(x_t_1_downsamp_1)/len(x_t_1_downsamp_1)
x_f_2_downsamp_1 = fft(x_t_2_downsamp_1)/len(x_t_2_downsamp_1)
x_f_3_downsamp_1 = fft(x_t_3_downsamp_1)/len(x_t_3_downsamp_1)
x_f_4_downsamp_1 = fft(x_t_4_downsamp_1)/len(x_t_4_downsamp_1)

x_f_total_downsamp_1[abs(x_f_total_downsamp_1)<1e-3] = np.nan
x_f_1_downsamp_1[abs(x_f_1_downsamp_1)<1e-3] = np.nan
x_f_2_downsamp_1[abs(x_f_2_downsamp_1)<1e-3] = np.nan
x_f_3_downsamp_1[abs(x_f_3_downsamp_1)<1e-3] = np.nan
x_f_4_downsamp_1[abs(x_f_4_downsamp_1)<1e-3] = np.nan


fig3 = go.Figure()
fig3.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_1_downsamp_1),
                           z=np.imag(x_f_1_downsamp_1),
                           mode='markers', name=f'Channel 1 Content in Stream 3 (delay=2)',
                           marker=dict(size=5, symbol="circle")))    
fig3.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_2_downsamp_1),
                           z=np.imag(x_f_2_downsamp_1),
                           mode='markers', name=f'Channel 2 Content in Stream 3 (delay=2)',
                           marker=dict(size=5, symbol="square")))  
fig3.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_3_downsamp_1),
                           z=np.imag(x_f_3_downsamp_1),
                           mode='markers', name=f'Channel 3 Content in Stream 3 (delay=2)',
                           marker=dict(size=5, symbol="diamond")))  
fig3.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_4_downsamp_1),
                           z=np.imag(x_f_4_downsamp_1),
                           mode='markers', name=f'Channel 4 Content in Stream 3 (delay=2)',
                           marker=dict(size=5, symbol="cross")))


fig3.update_layout(width=1000,height=800,
    scene=dict(
        xaxis=dict( title="Frequency Samples"),
        yaxis=dict(range=[-1.5, 1.5], title="Y"),
        zaxis=dict(range=[-1.5, 1.5], title="Z"),
        aspectmode="manual",           # allow manual control
        aspectratio=dict(x=1, y=0.5, z=0.5),  # scale of each axis
        camera=dict(eye=dict(x=-1, y=-1, z=1)),  # adjust the camera so labels are visible

    ),
    title="Decomposed Spectrum of Stream 3 (delay=2)"
)

# Zero axes

fig3.add_trace(go.Scatter3d(
    x=[0,1000], y=[0,0], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig3.add_trace(go.Scatter3d(
    x=[0,0], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig3.add_trace(go.Scatter3d(
    x=[0,0], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig3.add_trace(go.Scatter3d(
    x=[1000,1000], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig3.add_trace(go.Scatter3d(
    x=[1000,1000], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))


fig3.show()

In [ ]:
### downsample into 4 staggered streams
# stream 1
idx = np.arange(1000)
x_t_total_downsamp_1 = x_t_total[3::4]
x_t_1_downsamp_1 = x_t_1[3::4]
x_t_2_downsamp_1 = x_t_2[3::4]
x_t_3_downsamp_1 = x_t_3[3::4]
x_t_4_downsamp_1 = x_t_4[3::4]

x_f_total_downsamp_1 = fft(x_t_total_downsamp_1)/len(x_t_total_downsamp_1)
x_f_1_downsamp_1 = fft(x_t_1_downsamp_1)/len(x_t_1_downsamp_1)
x_f_2_downsamp_1 = fft(x_t_2_downsamp_1)/len(x_t_2_downsamp_1)
x_f_3_downsamp_1 = fft(x_t_3_downsamp_1)/len(x_t_3_downsamp_1)
x_f_4_downsamp_1 = fft(x_t_4_downsamp_1)/len(x_t_4_downsamp_1)

x_f_total_downsamp_1[abs(x_f_total_downsamp_1)<1e-3] = np.nan
x_f_1_downsamp_1[abs(x_f_1_downsamp_1)<1e-3] = np.nan
x_f_2_downsamp_1[abs(x_f_2_downsamp_1)<1e-3] = np.nan
x_f_3_downsamp_1[abs(x_f_3_downsamp_1)<1e-3] = np.nan
x_f_4_downsamp_1[abs(x_f_4_downsamp_1)<1e-3] = np.nan


fig4 = go.Figure()
fig4.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_1_downsamp_1),
                           z=np.imag(x_f_1_downsamp_1),
                           mode='markers', name=f'Channel 1 Content in Stream 4 (delay=3)',
                           marker=dict(size=5, symbol="circle")))    
fig4.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_2_downsamp_1),
                           z=np.imag(x_f_2_downsamp_1),
                           mode='markers', name=f'Channel 2 Content in Stream 4 (delay=3)',
                           marker=dict(size=5, symbol="square")))  
fig4.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_3_downsamp_1),
                           z=np.imag(x_f_3_downsamp_1),
                           mode='markers', name=f'Channel 3 Content in Stream 4 (delay=3)',
                           marker=dict(size=5, symbol="diamond")))  
fig4.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_4_downsamp_1),
                           z=np.imag(x_f_4_downsamp_1),
                           mode='markers', name=f'Channel 4 Content in Stream 4 (delay=3)',
                           marker=dict(size=5, symbol="cross")))


fig4.update_layout(width=1000,height=800,
    scene=dict(
        xaxis=dict( title="Frequency Samples"),
        yaxis=dict(range=[-1.5, 1.5], title="Y"),
        zaxis=dict(range=[-1.5, 1.5], title="Z"),
        aspectmode="manual",           # allow manual control
        aspectratio=dict(x=1, y=0.5, z=0.5),  # scale of each axis
        camera=dict(eye=dict(x=-1, y=-1, z=1)),  # adjust the camera so labels are visible

    ),
    title="Decomposed Spectrum of Stream 4 (delay=3)"
)

# Zero axes

fig4.add_trace(go.Scatter3d(
    x=[0,1000], y=[0,0], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig4.add_trace(go.Scatter3d(
    x=[0,0], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig4.add_trace(go.Scatter3d(
    x=[0,0], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig4.add_trace(go.Scatter3d(
    x=[1000,1000], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig4.add_trace(go.Scatter3d(
    x=[1000,1000], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))


fig4.show()

In [ ]:
# --- Combine them into one figure with frames ---
combined = go.Figure()

# Add the first figure's traces as the initial data
combined.add_traces(fig1.data)

# Build frames from all figs
frames = []
for i, f in enumerate([fig1, fig2, fig3, fig4], start=1):
    frames.append(go.Frame(data=f.data, name=f"frame{i}"))

combined.frames = frames

# Add slider
combined.update_layout(
    sliders=[{
        "steps": [
            {"args": [[f.name], {"frame": {"duration": 0}, "mode": "immediate"}],
             "label": f"Stream {i+1} (delay={i})", "method": "animate"}
            for i, f in enumerate(combined.frames)
        ],
        "transition": {"duration": 0},
        "x": 0.1, "y": -0.1, "len": 0.9
    }],
    width=1000,height=800,
    scene=dict(
        xaxis=dict( title="Frequency Samples"),
        yaxis=dict(range=[-1.5, 1.5], title="Y"),
        zaxis=dict(range=[-1.5, 1.5], title="Z"),
        aspectmode="manual",           # allow manual control
        aspectratio=dict(x=1, y=0.5, z=0.5),  # scale of each axis
        camera=dict(eye=dict(x=-1, y=-1, z=1)),  # adjust the camera so labels are visible

    ),
    title="Decomposed Spectrum of Staggered Down-sampled Streams"
)

combined.show()

In [ ]:
combined.write_html("staggered_downsampled_streams.html",
               include_plotlyjs='cdn',
               full_html=False)